In [2]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector as mysql

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

import numpy as np
import imutils
import time
import cv2
import os

In [3]:
def generate_dataset():
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result','Please provide complete details of the user')
    else:
        mydb=mysql.connect(
        host="localhost",
        user="root",
        passwd="12345",
        database="meetdb"
        )
        mycursor=mydb.cursor()
        mycursor.execute("SELECT * from face")
        myresult=mycursor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into face(id,name,age,address) values(%s,%s,%s,%s)"
        val=(id,t1.get(),t2.get(),t3.get())
        mycursor.execute(sql,val)
        mydb.commit()
        messagebox.showinfo('Result','Generating dataset completed!!!')

In [4]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)


    faces = []
    locs = []
    preds = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        if confidence > 0.5:
            
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            


            faces.append(face)
            locs.append((startX, startY, endX, endY))
            if len(faces) > 0:

                faces = np.array(faces, dtype="float32")
                preds = maskNet.predict(faces, batch_size=32)
    return (locs, preds)


def show():
    prototxtPath = r"face_detector\deploy.prototxt"
    weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
    maskNet = load_model("mask_detector.model")
    vs = VideoStream(src=0).start()
    print("[INFO] starting video stream...")
    while True:
        frame = vs.read()
        
        frame = imutils.resize(frame, width=1200)
        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
        for (box, pred) in zip(locs, preds):
            mydb=mysql.connect(
                host="localhost",
                user="root",
                passwd="12345",
                database="meetdb")
            mycursor=mydb.cursor()
            mycursor.execute("select name from face order by id desc limit 1")
            s = mycursor.fetchone()
            s = ''+''.join(s)
            s=str(s)
            
            (startX, startY, endX, endY) = box
            (mask, withoutMask) = pred
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            cv2.putText(frame, s, (startX, startY - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.putText(frame, label, (startX, startY - 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
    cv2.destroyAllWindows()
    vs.stop()
show()

[INFO] starting video stream...
(1, 1, 107, 7)
(1, 1, 104, 7)
(1, 1, 109, 7)
(1, 1, 113, 7)
(1, 1, 107, 7)
(1, 1, 113, 7)
(1, 1, 104, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 102, 7)
(1, 1, 102, 7)
(1, 1, 108, 7)
(1, 1, 106, 7)
(1, 1, 106, 7)
(1, 1, 109, 7)
(1, 1, 105, 7)
(1, 1, 104, 7)
(1, 1, 104, 7)
(1, 1, 111, 7)
(1, 1, 101, 7)
(1, 1, 109, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 108, 7)
(1, 1, 111, 7)
(1, 1, 108, 7)
(1, 1, 108, 7)
(1, 1, 109, 7)
(1, 1, 115, 7)
(1, 1, 112, 7)
(1, 1, 110, 7)
(1, 1, 114, 7)
(1, 1, 109, 7)
(1, 1, 111, 7)
(1, 1, 106, 7)
(1, 1, 108, 7)
(1, 1, 112, 7)
(1, 1, 113, 7)
(1, 1, 109, 7)
(1, 1, 107, 7)
(1, 1, 108, 7)
(1, 1, 111, 7)
(1, 1, 111, 7)
(1, 1, 111, 7)
(1, 1, 112, 7)
(1, 1, 111, 7)
(1, 1, 118, 7)
(1, 1, 115, 7)
(1, 1, 111, 7)
(1, 1, 114, 7)
(1, 1, 109, 7)
(1, 1, 113, 7)
(1, 1, 108, 7)
(1, 1, 109, 7)
(1, 1, 107, 7)
(1, 1, 113, 7)
(1, 1, 109, 7)
(1, 1, 110, 7)
(1, 1, 110, 7)
(1, 1, 112, 7)
(1, 1, 109, 7)
(1, 1, 107, 7)
(1, 1, 107, 7)
(1, 1, 108, 7)
(1, 1, 1

In [7]:
window=tk.Tk()
window.title("Face recognition system")
window.config(background="lightblue")
l1=tk.Label(window,text="Name",font=("Algerian",20))
l1.grid(column=0, row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1, row=0)
l2=tk.Label(window,text="Age",font=("Algerian",20))
l2.grid(column=0, row=2)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1, row=2)

l3=tk.Label(window,text="Address",font=("Algerian",20))
l3.grid(column=0, row=4)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1, row=4)

b1=tk.Button(window,text="Generate dataset",font=("Algerian",20),bg="pink",fg="black",command=generate_dataset)
b1.grid(column=1,row=10)





b2=tk.Button(window,text="detection",font=("Algerian",20),bg="pink",fg="black",command=show)
b2.grid(column=2,row=10)

window.geometry("700x400")
window.mainloop()

In [8]:
mydb=mysql.connect(
    host="localhost",
    user="root",
    passwd="12345",
    database="meetdb"
    )
mycursor=mydb.cursor()
id=1
mycursor.execute('SELECT name FROM face WHERE id='+str(id))
s = mycursor.fetchone()
s = ''+''.join(s)
print(s)

meet
